# Load Necessary Labries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from transformers import SegformerForSemanticSegmentation

# Define Transformation like (resize images, convert to tensor, normalize)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize the images
    transforms.ToTensor(),          # convert images to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normalize the images
])

# Path to the train and test datasets


In [ ]:
train_data_path = "dataset"
test_data_path = "dataset"

# Load train and test datasets using ImageFolder, which automatically assigns labels based on folder names


In [ ]:
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)

# Create data loaders


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Mapping of class labels (folder names) to indices


In [ ]:
class_names = train_dataset.classes  # Should be ['CN', 'EMCI', 'LMCI', 'MCI', 'AD']

# Check dataset structure and classes


In [ ]:
print(f"Class names: {class_names}")
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

# Access first batch of train data


In [ ]:
for images, labels in train_loader:
    print(f"Image batch shape: {images.size()}")
    print(f"Labels: {labels}")
    break

# U-Net Model Definition


In [ ]:
# U-Net Model for Padding and Enhancement (pretrained or custom model)
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet, self).__init__()

        def CBR(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )

        self.encoder1 = CBR(in_channels, 64)
        self.encoder2 = CBR(64, 128)
        self.encoder3 = CBR(128, 256)
        self.encoder4 = CBR(256, 512)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = CBR(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = CBR(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = CBR(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = CBR(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = CBR(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool(e1))
        e3 = self.encoder3(self.pool(e2))
        e4 = self.encoder4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        return self.final_conv(d1)


# Initialize U-Net


In [ ]:
model = UNet(in_channels=3, out_channels=3)

# Define transformation for dataset (resize and normalize)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define loss function and optimizer


In [ ]:
criterion = nn.MSELoss()  # Pixel-wise comparison using MSE loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)  # Loss comparing input and output images
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader)}")

In [ ]:
# Save the pretrained U-Net model (optional)
torch.save(model.state_dict(), 'unet_pretrained.pth')

In [ ]:
# Switch model to evaluation mode for enhancement
model.eval()

In [ ]:
# Function to enhance an image using U-Net
def enhance_image(image):
    with torch.no_grad():
        image = image.unsqueeze(0)  # Add batch dimension
        output = model(image)  # Apply U-Net for padding
        return output.squeeze(0)  # Remove batch dimension

# Apply U-Net enhancement on both training and testing datasets


In [ ]:
def process_and_display(loader, dataset_type):
    for images, _ in loader:
        image = images[0]  # Select the first image from the batch
        enhanced_image = enhance_image(image)

        # Convert tensors to numpy arrays for visualization
        original_image = image.permute(1, 2, 0).cpu().numpy()
        enhanced_image = enhanced_image.permute(1, 2, 0).cpu().numpy()

        # Plot original vs enhanced image
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].imshow(np.clip(original_image, 0, 1))
        ax[0].set_title(f"Original Image ({dataset_type})")

        ax[1].imshow(np.clip(enhanced_image, 0, 1))
        ax[1].set_title(f"Enhanced Image ({dataset_type})")

        plt.show()
        break  # Display only one image

# Enhance and display images from both training and testing datasets


In [ ]:
print("Processing Training Dataset:")
process_and_display(train_loader, "Training")

print("Processing Testing Dataset:")
process_and_display(test_loader, "Testing")

# Load the pretrained MDXNet model for noise reduction


In [ ]:
noise_reduction_model = MDXNet(in_channels=3, out_channels=3)

# Load the pretrained weights


In [ ]:
noise_reduction_model.load_state_dict(torch.load('mdxnet_pretrained.h5'))

# Switch MDXNet model to evaluation mode


In [ ]:
noise_reduction_model.eval()

# Function to reduce noise using the pretrained MDXNet


In [ ]:
def reduce_noise(image):
    with torch.no_grad():
        image = image.unsqueeze(0)  # Add batch dimension
        output = noise_reduction_model(image)  # Apply MDXNet for noise reduction
        return output.squeeze(0)  # Remove batch dimension


# Modify the process_and_display function to include noise reduction using the pretrained MDXNet


In [ ]:
def process_and_display_with_noise_reduction(loader, dataset_type):
    for images, _ in loader:
        image = images[0]  # Select the first image from the batch

        # Step 1: Apply noise reduction using pretrained MDXNet
        denoised_image = reduce_noise(image)

        # Step 2: Apply U-Net enhancement on the denoised image
        enhanced_image = enhance_image(denoised_image)

        # Convert tensors to numpy arrays for visualization
        original_image = image.permute(1, 2, 0).cpu().numpy()  # Original image
        denoised_image_np = denoised_image.permute(1, 2, 0).cpu().numpy()  # Denoised image
        enhanced_image_np = enhanced_image.permute(1, 2, 0).cpu().numpy()  # Enhanced image

        # Plot original, denoised, and enhanced images
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].imshow(np.clip(original_image, 0, 1))
        ax[0].set_title(f"Original Image ({dataset_type})")

        ax[1].imshow(np.clip(denoised_image_np, 0, 1))
        ax[1].set_title(f"Denoised Image ({dataset_type})")

        ax[2].imshow(np.clip(enhanced_image_np, 0, 1))
        ax[2].set_title(f"Enhanced Image ({dataset_type})")

        plt.show()
        break  # Display only one image


# Display Results

In [ ]:
print("Processing Training Dataset with Pretrained MDXNet for Noise Reduction:")
process_and_display_with_noise_reduction(train_loader, "Training")

print("Processing Testing Dataset with Pretrained MDXNet for Noise Reduction:")
process_and_display_with_noise_reduction(test_loader, "Testing")


# SRGAN for Image Sharpness Enhancement


In [ ]:
class SRGAN(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(SRGAN, self).__init__()

        # Define a simple upscaling network to enhance sharpness
        self.upscale = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        return self.upscale(x)

# Load the pretrained SRGAN model


In [ ]:
sharpness_model = SRGAN(in_channels=3, out_channels=3)
sharpness_model.load_state_dict(torch.load('srgan_pretrained.pth'))  # Pretrained model path
sharpness_model.eval()

# Function to enhance sharpness using SRGAN


In [ ]:
def enhance_sharpness(image):
    with torch.no_grad():
        image = image.unsqueeze(0)  # Add batch dimension
        output = sharpness_model(image)  # Apply SRGAN for sharpness enhancement
        return output.squeeze(0)  # Remove batch dimension


# Function to process and display images with noise reduction, U-Net, and SRGAN sharpness enhancement


In [ ]:
def process_and_display_full_pipeline(loader, dataset_type):
    for images, _ in loader:
        image = images[0]  # Select the first image from the batch

        # Apply noise reduction using pretrained MDXNet
        denoised_image = reduce_noise(image)

        #  Apply U-Net for padding and enhancement
        enhanced_image = enhance_image(denoised_image)

        # Apply SRGAN for sharpness enhancement
        sharp_image = enhance_sharpness(enhanced_image)

        # Convert tensors to numpy arrays for visualization
        original_image = image.permute(1, 2, 0).cpu().numpy()
        denoised_image = denoised_image.permute(1, 2, 0).cpu().numpy()
        enhanced_image = enhanced_image.permute(1, 2, 0).cpu().numpy()
        sharp_image = sharp_image.permute(1, 2, 0).cpu().numpy()

        # Plot original vs denoised vs enhanced vs sharp image
        fig, ax = plt.subplots(1, 4, figsize=(20, 5))
        ax[0].imshow(np.clip(original_image, 0, 1))
        ax[0].set_title(f"Original Image ({dataset_type})")

        ax[1].imshow(np.clip(denoised_image, 0, 1))
        ax[1].set_title(f"Denoised Image ({dataset_type})")

        ax[2].imshow(np.clip(enhanced_image, 0, 1))
        ax[2].set_title(f"Enhanced Image ({dataset_type})")

        ax[3].imshow(np.clip(sharp_image, 0, 1))
        ax[3].set_title(f"Sharp Image ({dataset_type})")

        plt.show()
        break  # Display only one image


# Apply the updated pipeline on training and testing datasets


In [ ]:
print("Processing Training Dataset with Full Pipeline:")
process_and_display_full_pipeline(train_loader, "Training")

print("Processing Testing Dataset with Full Pipeline:")
process_and_display_full_pipeline(test_loader, "Testing")

# Define RNet30 model for classification


In [ ]:
class RNet30(nn.Module):
    def __init__(self, num_classes=5):  #  5 classes: CN, EMCI, LMCI, MCI, AD
        super(RNet30, self).__init__()

        # Define convolutional layers
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )

        # Fully connected layers for classification
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        # Forward pass through convolutional layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Flatten the output for fully connected layers
        x = x.view(x.size(0), -1)

        # Forward pass through fully connected layers
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)

        return x


# Initialize the RNet30 model


In [ ]:
num_classes = 5  # Number of output classes (CN, EMCI, LMCI, MCI, AD)
rnet_model = RNet30(num_classes=num_classes)

# Define loss function and optimizer


In [ ]:
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(rnet_model.parameters(), lr=0.001)


# Training loop for RNet30


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    rnet_model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = rnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader)}, Accuracy: {accuracy}%")

# Save the trained RNet30 model
torch.save(rnet_model.state_dict(), 'rnet30_trained')

# Switch to evaluation mode for testing
rnet_model.eval()

# Testing loop for RNet30
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = rnet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy on the test dataset
test_accuracy = 100 * correct / total
print(f"Test Accuracy of RNet30: {test_accuracy}%")

# Load pretrained SegFormer model


In [ ]:
segformer = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b0-finetuned-ade-512-512')

# Switch to evaluation mode


In [ ]:
segformer.eval()

# Function to apply SegFormer for segmentation of affected regions


In [ ]:
def segment_brain_region(image):
    with torch.no_grad():
        # Preprocess the image for SegFormer
        image = image.unsqueeze(0)  # Add batch dimension

        # Apply SegFormer for segmentation
        outputs = segformer(image)['logits']  # Get segmentation logits
        segmentation_map = torch.argmax(outputs, dim=1)  # Get predicted class for each pixel

        return segmentation_map.squeeze(0)  # Remove batch dimension


# Updated pipeline for segmentation after classification


In [ ]:
def process_and_segment(loader, dataset_type):
    for images, labels in loader:
        image = images[0]  # Select the first image from the batch

        # Step 1: Classify using RNet30
        rnet_model.eval()  # Switch to evaluation mode
        outputs = rnet_model(image.unsqueeze(0))  # Add batch dimension
        _, predicted = torch.max(outputs.data, 1)

        # If the prediction indicates an affected brain region, proceed with segmentation
        if predicted.item() in [1, 2, 3, 4]:  #  1 to 4 represent EMCI, LMCI, MCI, SD
            print(f"Detected affected region in {dataset_type} image. Applying SegFormer for segmentation...")

            # Step 2: Apply SegFormer for segmentation
            segmentation_map = segment_brain_region(image)

            # Convert tensors to numpy arrays for visualization
            original_image = image.permute(1, 2, 0).cpu().numpy()
            segmentation_map = segmentation_map.cpu().numpy()

            # Plot original vs segmented image
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(np.clip(original_image, 0, 1))
            ax[0].set_title(f"Original Image ({dataset_type})")

            ax[1].imshow(segmentation_map, cmap='gray')
            ax[1].set_title(f"Segmented Affected Region ({dataset_type})")

            plt.show()

        else:
            print(f"No affected region detected in {dataset_type} image.")

        break  # Process only one image

# Apply the full pipeline on training and testing datasets


In [ ]:
print("Processing and Segmenting Training Dataset:")
process_and_segment(train_loader, "Training")

print("Processing and Segmenting Testing Dataset:")
process_and_segment(test_loader, "Testing")